In [42]:
import torch
from data_loader import DatasetPipline
from torch.utils.data import Dataset, DataLoader
from model import U2NET
from callbacks import EarlyStopping, IOU
from torch.autograd import Variable
import torch.nn as nn
import pprint
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time

bce_loss = nn.BCELoss(size_average=True)

/home/mbarakat/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [43]:
def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels_v):

    loss0 = bce_loss(d0,labels_v)
    loss1 = bce_loss(d1,labels_v)
    loss2 = bce_loss(d2,labels_v)
    loss3 = bce_loss(d3,labels_v)
    loss4 = bce_loss(d4,labels_v)
    loss5 = bce_loss(d5,labels_v)
    loss6 = bce_loss(d6,labels_v)

    loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
    # print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

    return loss0, loss

In [44]:
net = U2NET(1, 1)

In [45]:
train_ = DatasetPipline(src="data", split="Train")
train = DataLoader(train_, batch_size=12, shuffle=True, num_workers=1)


# test iou=1

In [46]:
running_loss = 0.0
running_tar_loss = 0.0
iou = IOU()
loss2 = 0
loss = 0

In [47]:
for i, data in enumerate(train):
    net.train(mode=False)
    inputs, labels = data['image'], data['label']

    inputs = inputs.type(torch.FloatTensor)
    labels = labels.type(torch.FloatTensor)
    
    
    inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)
#     net.cuda()
    
    
    d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
    break


/home/mbarakat/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/mbarakat/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/mbarakat/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [48]:
iou.calc(d0, d0)

In [49]:
iou.mean()

1.0

# test that target is 0 or 1

In [50]:
# sum_max = 0
# for i, data in enumerate(train):
#     inputs, labels = data['image'], data['label']
# #     print(labels.shape)
#     labels[labels==1]=0
# #     print(labels.max())
# #     print(labels.shape)
#     sum_max += labels.max()
    
# assert sum_max == 0

In [51]:
%matplotlib widget
# %matplotlib inline

In [52]:
def draw(x, y, y_hat=None):
    x = np.squeeze(x)
    x = np.expand_dims(x, -1)
    
    y = np.squeeze(y)
    y = np.expand_dims(y, -1)
    
    x = x*255
    x = cv2.cvtColor(x, cv2.COLOR_GRAY2RGB)
#     print(x.shape)
    
    y = y*255
    
    y = y.astype(np.uint8)
    
    

    contours, hierarchy = cv2.findContours(y, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x = cv2.drawContours(x, contours, -1, (0,255,0), 1)
    
#     if y_hat = None:
    y_hat = np.squeeze(y_hat)
    y_hat = np.expand_dims(y_hat, -1)
    y_hat = y_hat*255
    y_hat = y_hat.astype(np.uint8)
    contours, hierarchy = cv2.findContours(y_hat, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x = cv2.drawContours(x, contours, -1, (255,0,0), 1)
        
    return x

# i, data = next(enumerate(val))
# inputs, labels = data['image'], data['label']

# inputs_, labels_ = inputs.numpy()[0], labels.numpy()[0]

# result = draw(inputs_, labels_).astype(np.int32)
# plt.figure(figsize = (8,8))
# plt.imshow(result)
# plt.axis('off')
# # plt.text(40, -3, "Red: y_hat", fontsize="x-large", c="red")
# plt.text(180, -3, "Green: y", fontsize="x-large", c="green")
# plt.title("Segmentation-%d", pad=15)
# plt.show()

In [53]:
val_ = DatasetPipline(src="data", split="Validation")
val = DataLoader(val_, batch_size=4, shuffle=True, num_workers=1)
x = 2
y = 2
i=0
current_x = 0 
current_y = 0 

fig = plt.figure(figsize=(15,15), constrained_layout=True)
axs = fig.subplots(x, y)

# plt.text(40, -3, "Red: y_hat", fontsize="x-large", c="red")
# plt.text(180, -3, "Green: y", fontsize="x-large", c="green")
# plt.title("Segmentation", pad=15)

batch = next(enumerate(val))[1]
input_, label = batch['image'], batch['label']

net = U2NET(1, 1, True)
PATH = "/home/mbarakat/U-2-Net/saved_models/expeiraments/12/_bce_itr_14000_train_0.045961_tar_0.002637_val_loss_0.089783_val_iou_0.800485.pth"

load = torch.load(PATH)
net.load_state_dict(load)

net.train(mode=False)
net.to(torch.device("cpu"))
    
for data in zip(input_, label):
    
    inputs, labels = data[0], data[1]
    
    inputs = torch.unsqueeze(inputs, 0)
    labels = torch.unsqueeze(labels, 0)
    
    inputs = inputs.type(torch.FloatTensor)
    labels = labels.type(torch.FloatTensor)


    inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

    iou = IOU()

    d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
    loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels)
    iou.calc(d0, labels)
#     print("loss2: ",loss2.item(), "\nloss: ",loss.item(), "\niou: ", iou.mean())



    inputs_, d1_ = inputs.numpy()[0], d0.detach().numpy()[0]

    result = draw(inputs_, labels, d1_).astype(np.int32)
    
    ax=axs[current_x][current_y]
    ax.imshow(result)
    ax.axis('off')
#     ax.text = 
    
    
    if (current_x+1) % x == 0 and current_y !=0:
        current_y = current_y+1 
        current_y = current_y % y 
        
    elif current_x == x-1 and current_y ==0:
        current_y = current_y+1 

#     print("current_x ", current_x, "current_y ", current_y)
        
    current_x = current_x+1 
    current_x = current_x % x
    
        
    
#     current_y = current_y+1 if current_y < y else 0    
    


plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
def show_idx(idx):
    val_ = DatasetPipline(src="data", split="Validation")[idx]

    inputs, labels = torch.from_numpy(val_['image']), torch.from_numpy(val_['label'])

    inputs = torch.unsqueeze(inputs, 0)
    labels = torch.unsqueeze(labels, 0)

    inputs = inputs.type(torch.FloatTensor)
    labels = labels.type(torch.FloatTensor)

    net = U2NET(1, 1, True)
    PATH = "/home/mbarakat/U-2-Net/saved_models/expeiraments/12/_bce_itr_14000_train_0.045961_tar_0.002637_val_loss_0.089783_val_iou_0.800485.pth"

    load = torch.load(PATH)
    net.load_state_dict(load)

    net.train(mode=False)
    net.to(torch.device("cpu"))
#     print(net.)
    inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

    iou = IOU()
    
    t1 = time.time()
    d0, d1, d2, d3, d4, d5, d6 = net(inputs_v)
    print(time.time()-t1)
    loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels)
    iou.calc(d0, labels)
    #     print("loss2: ",loss2.item(), "\nloss: ",loss.item(), "\niou: ", iou.mean())



    inputs_, d1_ = inputs.numpy()[0], d0.detach().numpy()[0]

    result = draw(inputs_, labels, d1_).astype(np.int32)
    
#     fig.canvas.toolbar_visible = False
#     fig.canvas.header_visible = False
#     fig.canvas.resizable = True

    plt.figure(figsize = (8,8))
    plt.imshow(result)
    plt.axis('off')
    plt.text(40, -3, "Red: y_hat", fontsize="x-large", c="red")
    plt.text(180, -3, "Green: y", fontsize="x-large", c="green")
    plt.title("Segmentation-%d"%idx, pad=15)
#     plt.ion()
    return
    plt.show()



In [55]:
show_idx(idx=21)

0.08653044700622559


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
d0 = (torch.ones_like(labels))-1
d0 = Variable(d0, requires_grad=False)

iou = IOU()


loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, labels)
iou.calc(d0, labels)
print("loss2: ",loss2.item(), "\nloss: ",loss.item(), "\niou: ", iou.mean())


loss2:  0.0 
loss:  0.04007094353437424 
iou:  1.0


# test that loss of same input = 0

In [61]:
# d0_ = d0.d
d0_ = d0.type(torch.FloatTensor)
d0_ = Variable(d0_, requires_grad=False)

loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d0_)

print("loss2: ",loss2.item(), "\nloss: ",loss.item())


loss2:  0.6918859481811523 
loss:  4.852900981903076


In [62]:
d0_ = d0.type(torch.FloatTensor)
d0_ = Variable(d0_, requires_grad=False)

d0_ = (d0_>0.5).float()
d0 = (d0>0.5).float()


loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d0_)

print("loss2: ",loss2.item(), "\nloss: ",loss.item())

loss2:  0.0 
loss:  4.193838119506836


In [63]:
y_true = torch.ones_like(d0)
y_true = Variable(y_true, requires_grad=False)

loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, y_true)

print("loss2: ",loss2.item(), "\nloss: ",loss.item())


loss2:  0.0 
loss:  4.193838119506836


In [41]:
d0.max()

tensor(0.5996, device='cuda:0', grad_fn=<MaxBackward1>)

In [42]:
d0.min()

tensor(0.5929, device='cuda:0', grad_fn=<MinBackward1>)

In [57]:
loss2, loss = muti_bce_loss_fusion(y_true, d1, d2, d3, d4, d5, d6, y_true)

print("loss2: ",loss2.item(), "\nloss: ",loss.item())


loss2:  0.0 
loss:  4.1920318603515625


In [53]:
loss2.data.item()

0.0

In [67]:
y_true = torch.ones_like(d0)
y_true = Variable(y_true.cuda(), requires_grad=False)

d0 = (torch.ones_like(d0))-1
d0 = Variable(d0.cuda(), requires_grad=False)

loss2 = bce_loss(d0, y_true)

print("loss2: ",loss2.item())

loss2:  100.0


In [48]:
print(net)

U2NET(
  (stage1): RSU7(
    (rebnconvin): REBNCONV(
      (conv_s1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (rebnconv1): REBNCONV(
      (conv_s1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv2): REBNCONV(
      (conv_s1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn_s1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu_s1): ReLU(inplace=True)
    )
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (rebnconv3): REBNCONV(
      (conv_s1): Conv2d(32, 32, k

In [52]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/test')

writer.add_graph(net, inputs)
writer.close()
